# CSGO Win Probability Model Tuning
##### Peter Xenopoulos
##### January 31, 2020
This Jupyter notebook contains the code for model tuning. You will find the model fitting and tuning procedures for (1) Logistic Regression, (2) XGBoost and (3) CatBoost. Final models are saved to the `models` directory. You can load models using the `.load_model()` method. The data in the `data/` directory is an example of a few matches.

In [16]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss, accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

from catboost import Pool, CatBoost, CatBoostClassifier, cv

First we define our random seed and our cutoff date. Matches before and on the cutoff date comprise the training set. Matches after comprise the test set.

We also create two lists of columns. The first, without the `_CB` in the name, indicates variables that have been one hot encoded. This is necessary for XGBoost and Logistic Regression.

Finally, we read in our data.

In [2]:
# Set constants
RANDOM_STATE = 2020
CUTOFF_DATE = "2019-12-15"

# One Hot Encoded columns
COLS_ALL = ["MapName_de_dust2", "MapName_de_inferno", "MapName_de_mirage", "MapName_de_nuke", "MapName_de_overpass", "MapName_de_train", "MapName_de_vertigo",
       "TicksSinceStart", "CTEqVal", "TEqVal", "TRemaining", "CTRemaining", "THpRemaining", "CTHpRemaining", "BombPlanted", "CTDistBombsiteA", "CTDistBombsiteB", "TDistBombsiteA", "TDistBombsiteB"]
COLS_NO_SPATIAL = ["MapName_de_dust2", "MapName_de_inferno", "MapName_de_mirage", "MapName_de_nuke", "MapName_de_overpass", "MapName_de_train", "MapName_de_vertigo",
       "TicksSinceStart", "CTEqVal", "TEqVal", "TRemaining", "CTRemaining", "THpRemaining", "CTHpRemaining", "BombPlanted"]

# Non-encoded columns
COLS_CB_ALL = ["MapName", "TicksSinceStart", "CTEqVal", "TEqVal", "TRemaining", "CTRemaining", "THpRemaining", "CTHpRemaining", "BombPlanted", "CTDistBombsiteA", "CTDistBombsiteB", "TDistBombsiteA", "TDistBombsiteB"]
COLS_CB_NO_SPATIAL = ["MapName", "TicksSinceStart", "CTEqVal", "TEqVal", "TRemaining", "CTRemaining", "THpRemaining", "CTHpRemaining", "BombPlanted"]

# Read data
df = pd.read_csv("data/example_matches.csv")

Then, we preprocess the data, creating some one hot encoded columns. Next, we split into the test and train sets.

In [4]:
# Preprocess columns
df["GameDate"] = pd.to_datetime(df["GameDate"])
map_names = df["MapName"]
df = pd.get_dummies(df, columns = ["MapName"], drop_first = False)
df["MapName"] = map_names
#df = pd.get_dummies(df, columns = ["BombSite"], drop_first = False)
#df.drop(columns = ["BombSite_NotPlanted"], inplace = True)

# Train/Test split
train_df = df[df["GameDate"] < CUTOFF_DATE]
test_df = df[df["GameDate"] >= CUTOFF_DATE]

Now, we split the data into the various design matrices, as well as dropping NAs. 

In [5]:
# Data used in logistic regression and XGBoost
train_df.dropna(subset = COLS_ALL, inplace=True)
X_train_ALL = train_df[COLS_ALL]
X_train_NO_SPATIAL = train_df[COLS_NO_SPATIAL]
y_train = train_df["CTWin"]

test_df.dropna(subset = COLS_ALL, inplace=True)
X_test_ALL = test_df[COLS_ALL]
X_test_NO_SPATIAL = test_df[COLS_NO_SPATIAL]
y_test = test_df["CTWin"]

# Data used in CatBoost
train_df.dropna(subset = COLS_CB_ALL, inplace=True)
X_train_cb_ALL = train_df[COLS_CB_ALL]
X_train_cb_NO_SPATIAL = train_df[COLS_CB_NO_SPATIAL]
y_train_cb = train_df["CTWin"]

test_df.dropna(subset = COLS_CB_ALL, inplace=True)
X_test_cb_ALL = test_df[COLS_CB_ALL]
X_test_cb_NO_SPATIAL = test_df[COLS_CB_NO_SPATIAL]
y_test_cb = test_df["CTWin"]

/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/pxenopoulo

###### Print Function
We use this print function to print the train/test results of our best models.

In [6]:
def print_results(y_true_labels, y_pred_probs):
    """ Presents performance info
    """
    print("---------- LOG LOSS")
    print(log_loss(y_true_labels, y_pred_probs))
    print("---------- BRIER SCORE")
    print(brier_score_loss(y_true_labels, y_pred_probs[:,1]))
    print("---------- AUC")
    print(roc_auc_score(y_true_labels, y_pred_probs[:,1]))
    print("---------- ACCURACY")
    print(accuracy_score(y_true_labels, y_pred_probs[:,1] >= 0.5))

### Baseline Win Rate Results

This model uses the mean CT win percentage as its prediction.

In [8]:
test_df["BaselinePred"] = train_df.CTWin.mean()
print_results(test_df["CTWin"], np.column_stack((1 - test_df["BaselinePred"], test_df["BaselinePred"])))

/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


---------- LOG LOSS
0.6939758480193727
---------- BRIER SCORE
0.2504143067921506
---------- AUC
0.5
---------- ACCURACY
0.4729157960196518


### Map Average Results

This model uses the mean CT win percentage, by map, as its prediction.

In [9]:
def baseline_map(train, test):
    """ Create a baseline map performance
    """
    # Generate map win percentages
    map_win_rate = train.groupby("MapName").CTWin.mean().reset_index()
    map_win_rate.columns = ["MapName", "PredWinRate"]
    # Can't do a big join locally, so break it up by map
    test["PredMapWinRate"] = 0.5
    map_subset_df = []
    for map_name in test["MapName"].unique():
        subset_map = test[test["MapName"] == map_name]
        subset_map["PredMapWinRate"] = map_win_rate[map_win_rate["MapName"] == map_name].PredWinRate.values[0]
        map_subset_df.append(subset_map)
    test = pd.concat(map_subset_df)
    print_results(test["CTWin"], np.column_stack((1 - test["PredMapWinRate"], test["PredMapWinRate"])))

In [10]:
baseline_map(train_df, test_df)

/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


---------- LOG LOSS
0.69507021638641
---------- BRIER SCORE
0.25095784919869124
---------- AUC
0.5157164845053701
---------- ACCURACY
0.4852771506897524


### Logistic Regression

Here we present the logistic regression results on two feature sets, one using all features available and the other using no spatial features.

In [12]:
lr_all = LogisticRegression(random_state=2020).fit(X_train_ALL, y_train)
lr_all_test_probs = lr_all.predict_proba(X_test_ALL)

print_results(y_test, lr_all_test_probs)

/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


---------- LOG LOSS
0.5562906469600253
---------- BRIER SCORE
0.19179454216780603
---------- AUC
0.7749967100788642
---------- ACCURACY
0.6962260076343663


In [13]:
lr_no_spatial = LogisticRegression(random_state=2020).fit(X_train_NO_SPATIAL, y_train)
lr_no_spatial_test_probs = lr_no_spatial.predict_proba(X_test_NO_SPATIAL)

print_results(y_test, lr_no_spatial_test_probs)

/Users/pxenopoulos/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


---------- LOG LOSS
0.5707958001623034
---------- BRIER SCORE
0.19715591697605758
---------- AUC
0.7631896975936451
---------- ACCURACY
0.6887905988118542


### XGBoost

Here we present the XGBoost results on two feature sets, one using all features available and the other using no spatial features.

We search over the parameter space below, using grid search with 5-fold cross validation and a log loss scoring metric.

In [16]:
xgb_params = { 
 "max_depth"        : [6, 8, 10, 12, 14],
 "colsample_bytree" : [0.2, 0.4, 0.6, 0.8],
 "learning_rate"    : [0.01, 0.05, 0.1, 0.2],
 "min_child_weight" : [1, 3, 5, 7]}

xgb_kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
xgb = XGBClassifier(n_estimators=100, objective="logloss", tree_method="gpu_hist", gpu_id=0)

xgb_cv = GridSearchCV(xgb, param_grid=xgb_params, cv=xgb_kfold, scoring="neg_log_loss", verbose=True)

In [ ]:
xgb_cv.fit(X_train_ALL, y_train)
xgb_all = xgb_cv.best_estimator_
xgb_all.save_model("models/xgboost_all_features.model")
print(xgb_cv.best_params_)
xgb_all_test_probs = xgb_all.predict_proba(X_test_ALL)
print_results(y_test, xgb_all_test_probs)

In [ ]:
xgb_cv.fit(X_train_NO_SPATIAL, y_train)
xgb_no_spatial = xgb_cv.best_estimator_
xgb_no_spatial.save_model("models/xgboost_no_spatial_features.model")
print(xgb_cv.best_params_)
xgb_no_spatial_test_probs = xgb_no_spatial.predict_proba(X_test_NO_SPATIAL)
print_results(y_test, xgb_no_spatial_test_probs)

### CatBoost

Here we present the XGBoost results on two feature sets, one using all features available and the other using no spatial features.

We search over the parameter space below, using grid search with 5-fold cross validation and a log loss scoring metric.

In [ ]:
cb_params = {
    "learning_rate": [0.05, 0.1, 0.5, 1],
    "depth": [6, 8, 10, 12, 14],
    "l2_leaf_reg": [1, 3, 5, 7, 9]
}

cb_kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
cb_all_pool = Pool(data = X_train_cb_ALL, label = y_train_cb, cat_features = [0])
cb_no_spatial_pool = Pool(data = X_train_cb_NO_SPATIAL, label = y_train_cb, cat_features = [0])

cb = CatBoostClassifier(iterations=100, task_type="GPU", devices="0:1", custom_metric=["Logloss"])

cb_cv = GridSearchCV(cb, param_grid=cb_params, cv=cb_kfold, scoring="neg_log_loss", verbose=True)

In [ ]:
cb_cv.fit(X_train_cb_ALL, y_train, cat_features = [0])
cb_all = cb_cv.best_estimator_
cb_all.save_model("models/catboost_all_features.model", "cbm")
print(cb_cv.best_params_)
cb_all_test_probs = cb_all.predict_proba(X_test_cb_ALL)
print_results(y_test, cb_all_test_probs)

In [ ]:
cb_cv.fit(X_train_cb_NO_SPATIAL, y_train, cat_features = [0])
cb_no_spatial = cb_cv.best_estimator_
cb_no_spatial.save_model("models/catboost_no_spatial_features.model", "cbm")
print(cb_cv.best_params_)
cb_no_spatial_test_probs = cb_no_spatial.predict_proba(X_test_cb_NO_SPATIAL)
print_results(y_test, cb_no_spatial_test_probs)

In [ ]:
add bombsite, fix col names and cat features